In [1]:
"""
Tool-augmented fact-checking implementation using ReACT reasoning with external evidence.

This script implements the experimental approach where the LLM makes fact-checking
decisions using external evidence retrieved from Wikipedia. The agent
uses ReACT (Reasoning + Acting) pattern to iteratively reason about claims, retrieve
relevant evidence, and make informed judgments.

The tool-augmented approach:
1. Loads claims from the processed binary dataset
2. Uses ReACT agent to reason about each claim
3. Retrieves external evidence via Wikipedia API
4. Makes evidence-based True/False judgments
5. Records predictions, justifications, and evidence used
6. Saves results for comparison with baseline approach

This serves as the experimental condition to measure the effectiveness of
tool-augmented reasoning vs. relying solely on LLM internal knowledge.
"""

'\nTool-augmented fact-checking implementation using ReACT reasoning with external evidence.\n\nThis script implements the experimental approach where the LLM makes fact-checking\ndecisions using external evidence retrieved from Wikipedia. The agent\nuses ReACT (Reasoning + Acting) pattern to iteratively reason about claims, retrieve\nrelevant evidence, and make informed judgments.\n\nThe tool-augmented approach:\n1. Loads claims from the processed binary dataset\n2. Uses ReACT agent to reason about each claim\n3. Retrieves external evidence via Wikipedia API\n4. Makes evidence-based True/False judgments\n5. Records predictions, justifications, and evidence used\n6. Saves results for comparison with baseline approach\n\nThis serves as the experimental condition to measure the effectiveness of\ntool-augmented reasoning vs. relying solely on LLM internal knowledge.\n'

In [2]:
!pip install -U langchain langchain_community openai wikipedia requests pandas tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import pandas as pd
import sys
from typing import List, Dict, Optional
import logging
from tqdm import tqdm
import json
import warnings
warnings.filterwarnings('ignore')

In [5]:
os.chdir('/content/drive/MyDrive/DS301_Final_Project')

In [6]:
from src.utils import FactCheckingAgent, parse_llm_response, load_prompt, save_results
from tools.wikipedia_tool import WikipediaTool
from tools.newsapi_tool import NewsAPITool

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Clear existing handlers and force reconfigure
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.INFO,
                    format='%(levelname)s:%(name)s:%(message)s',
                    handlers=[
                        logging.StreamHandler(sys.stdout)
                    ],
                    force=True)
logger = logging.getLogger(__name__)

In [8]:
class ToolAugmentedFactChecker:
  """
  Tool-augmented fact-checker using ReACT agent with external evidence retrieval.

  This class implements the experimental approach that combines:
  1. Large Language Model reasoning capabilities
  2. External evidence retrieval from multiple sources
  3. ReACT pattern for iterative reasoning and acting
  4. Structured decision-making with justifications

  The key innovation is that instead of relying solely on the LLM's training data,
  the agent actively searches for current, relevant evidence before making judgments,
  and additionally reasons through this evidence in a systematic way.
  """
  def __init__(
                self,
                model_name: str='gpt-3.5-turbo',
                news_api_key: Optional[str]=None, # is optionally defined, and type string
                max_evidence_pieces: int=3 # This is for each tool
              ):
    """
    Initialize the tool-augmented fact-checker with external tools.

    Args:
        model_name: Name of the LLM model for the ReACT agent
        news_api_key: Optional API key for news source access
        max_evidence_pieces: Maximum evidence pieces to retrieve per claim
    """

    self.model_name = model_name

    # Initialize evidence retrieval tools
    logger.info("Initializing evidence retrieval tools...")

    # Wikipedia tool (always available, no API key required)
    self.wikipedia_tool = WikipediaTool(
            max_results=max_evidence_pieces,
            max_chars=400  # Limit length to avoid token limits
    )
    logger.info("Wikipedia tool initialized")

    # News API tool (optional, requires API key)
    if news_api_key:
      self.news_tool = NewsAPITool(
          api_key=news_api_key,
          max_results=max_evidence_pieces,
          max_chars=400)
      logger.info('News API tool initialized')
    else:
      self.news_tool=None
      logger.info('News API tool skipped - no API key provided')

    # Initialize the ReACT agent with access to evidence tools
    logger.info(f"Initializing ReACT agent with {model_name}...")
    try:
      self.agent = FactCheckingAgent(
        wikipedia_tool=self.wikipedia_tool,
        news_tool=self.news_tool,
        llm_model=model_name
      )
      logger.info('ReACT agent initialized successfully')
    except Exception as e:
      logger.info(f'Failed to initialize ReACT agent: {e}')
      raise # Can't proceed without it

  def check_single_claim(self, claim: str, claim_id: str = None) -> Dict[str, str]:
    """
        Fact-check a single claim using tool-augmented ReACT reasoning.

        This method implements the core experimental approach:
        1. Present the claim to the ReACT agent
        2. Agent reasons about what evidence is needed
        3. Agent uses tools to retrieve relevant evidence
        4. Agent analyzes evidence and makes judgment
        5. Returns structured result with evidence trail

        The key difference from baseline is that the agent actively gathers
        external evidence, as well as reasons about it, rather than relying on
        potentially outdated training data.

        Args:
            claim: The factual statement to verify
            claim_id: Optional identifier for tracking this claim

        Returns:
            Dictionary containing prediction, justification, evidence, and metadata
    """
    try:
      # Use ReACT agent to fact-check with evidence retrieval
      # The agent will automatically:
      # 1. Reason about the claim
      # 2. Decide what evidence to search for
      # 3. Use evidence_retrieval tool
      # 4. Analyze the evidence
      # 5. Provide structured judgment

      agent_result = self.agent.fact_check_with_reasoning(claim)

      # Extract evidence used during the reasoning process
      evidence_used = self._extract_evidence_from_agent_execution(claim) #

      result = {
        'claim_id': claim_id,
        'claim': claim,
        'prediction': agent_result['prediction'],
        'justification': agent_result['justification'],
        'evidence_used': evidence_used, # list
        'method': 'tool_augmented',
        'model': self.model_name,
        'tools_available': self._get_available_tools_info()
      }
      return result
    except Exception as e:
      # Handle individual claim failures gracefully
      logger.error(f"Error processing claim {claim_id}: {e}")
      error_result = {
          'claim_id': claim_id,
          'claim': claim,
          'prediction': 'Error',
          'justification': f'Tool-augmented processing failed: {str(e)}',
          'evidence_used': [],
          'method': 'tool_augmented',
          'model': self.model_name,
          'error': str(e)
      }
      return error_result

  def _extract_evidence_from_agent_execution(self, claim: str) -> List[Dict[str, str]]:
    """
      Extract evidence that was actually used during agent reasoning.

      This method attempts to capture what evidence the agent retrieved
      and used in its decision-making process. This is valuable for:
      1. Understanding agent reasoning process
      2. Evaluating evidence quality
      3. Debugging agent behavior

      Args:
          claim: The claim that was fact-checked

      Returns:
          List of evidence dictionaries with source and content
    """

    evidence_pieces = []
    try:
      # Get evidence that would be retrieved for this claim
      # Note: This is a simplified approximation since we can't easily
      # intercept the exact evidence used during agent execution
      wiki_evidence = self.wikipedia_tool.search_evidence(claim)
      evidence_pieces.extend(wiki_evidence)
      if self.news_tool:
        news_evidence = self.news_tool.search_evidence(claim)
        evidence_pieces.extend(news_evidence)
    except Exception as e:
      logger.warning(f'Could not extract evidence for claim: {e}')
    return evidence_pieces[:5]

  def _get_available_tools_info(self):
    """
    Get information about which tools are available to the agent.

    Returns:
        Dictionary indicating which evidence sources are available
    """
    return {
        'wikipedia': True,
        'news_api': False # Was using it previously but not anymore
    }

  def check_multiple_claims(self, claims_df: pd.DataFrame, max_samples: int = None) -> List[Dict]:
    """
    Fact-check multiple claims using tool-augmented approach with progress tracking.

    This method processes claims in batch using the ReACT agent for each one.
    It's designed to be robust to individual failures while providing comprehensive
    logging and progress updates.

    Note: This will be slower than baseline due to external API calls for evidence
    retrieval, but should provide more accurate and up-to-date fact-checking.

    Args:
        claims_df: DataFrame with 'id', 'statement', and 'binary_label' columns
        max_samples: Optional limit on number of claims to process

    Returns:
        List of result dictionaries with evidence and reasoning details
    """

    results=[]

    # Optionally limit samples for testing or resource constraints
    if max_samples:
        claims_df = claims_df.head(max_samples)
        logger.info(f"Processing first {max_samples} claims for testing")

    # Process each claim with detailed progress tracking
    for idx, row in tqdm(claims_df.iterrows(),
                        total=len(claims_df),
                        desc="Tool-augmented fact-checking"):
        try:
            # Process individual claim with tool-augmented reasoning
            result = self.check_single_claim(
                claim=row['statement'],
                claim_id=row.get('id', str(idx))
            )

            # Add ground truth for evaluation
            result['ground_truth'] = row['binary_label']
            results.append(result)

            # Log progress every 10 claims
            if (idx + 1) % 10 == 0:
              successful = len([r for r in results if r['prediction'] != 'Error'])
              logger.info(f'Processed {idx + 1} claims, {successful} successful')

        except Exception as e:
          # Log error but continue processing
          logger.error(f"Fatal error processing claim {idx}: {e}")
          # Create error result to maintain dataset alignment
          error_result = {
              'claim_id': row.get('id', str(idx)),
              'claim': row['statement'],
              'prediction': 'Error',
              'justification': f'Fatal processing error: {str(e)}',
              'evidence_used': [],
              'ground_truth': row['binary_label'],
              'method': 'tool_augmented',
              'model': self.model_name,
              'error': str(e)
          }
          results.append(error_result)

    logger.info(f"Completed tool-augmented fact-checking. Processed {len(results)} claims")
    return results




In [9]:
def run_tool_augmented_experiment(
  train_data_path: str = "/content/drive/MyDrive/DS301_Final_Project/data/train_binary.csv",
  test_data_path: str = "/content/drive/MyDrive/DS301_Final_Project/data/test_binary.csv",
  output_path: str = "/content/drive/MyDrive/DS301_Final_Project/results/tool_augmented_preds.jsonl",
  max_samples: int = None,
  model_name: str = "gpt-3.5-turbo",
  news_api_key: Optional[str]=None
):
  """
  Run the complete tool-augmented fact-checking experiment.
  This function orchestrates the entire tool-augmented experiment workflow:
    1. Load official LIAR dataset splits
    2. Initialize ReACT agent with evidence retrieval tools
    3. Process test set with tool-augmented reasoning
    4. Save detailed results including evidence trails
    5. Generate summary statistics for comparison

   Args:
        train_data_path: Path to training data (for reference)
        test_data_path: Path to test data (what we evaluate on)
        output_path: Where to save prediction results
        max_samples: Optional limit on test samples
        model_name: LLM model for ReACT agent
        news_api_key: Optional API key for news sources
  """

  logger.info("="*60)
  logger.info("STARTING TOOL-AUGMENTED FACT-CHECKING EXPERIMENT")
  logger.info("="*60)

  # Load the official LIAR dataset splits
  logger.info("Loading LIAR dataset splits...")

  try:
      # Load training data (just for statistics and reference)
      train_df = pd.read_csv(train_data_path)
      logger.info(f"Loaded training data: {len(train_df)} samples")
      # Load test data (this is what we'll actually evaluate on)
      test_df = pd.read_csv(test_data_path)
      logger.info(f"Loaded test data: {len(test_df)} samples")
  except FileNotFoundError as e:
      logger.error(f"Dataset file not found: {e}")
      logger.info("Please ensure both train_binary.csv and test_binary.csv exist in your data/ folder")
      return

  # Display dataset statistics for context
  logger.info("\nDataset Statistics:")
  logger.info(f"Training set label distribution:")
  logger.info(train_df['binary_label'].value_counts())
  logger.info(f"Test set label distribution:")
  logger.info(test_df['binary_label'].value_counts())

  # Initialize the tool-augmented fact-checker
  logger.info(f"\nInitializing tool-augmented fact-checker...")
  logger.info("This will set up ReACT agent with evidence retrieval capabilities")

  try:
      fact_checker = ToolAugmentedFactChecker(
          model_name=model_name,
          news_api_key=news_api_key
      )
  except Exception as e:
      logger.error(f"Failed to initialize tool-augmented fact-checker: {e}")
      logger.info("Check your API keys and internet connection")
      return

  # Run tool-augmented fact-checking on test set
  logger.info("\nStarting tool-augmented fact-checking process...")

  results = fact_checker.check_multiple_claims(test_df, max_samples=max_samples)

  # Save comprehensive results
  logger.info(f"Saving results to {output_path}")
  save_results(results, output_path)

  # Generate summary statistics
  logger.info("="*70)
  logger.info("TOOL-AUGMENTED EXPERIMENT SUMMARY")
  logger.info("="*70)

  successful_preds = [r for r in results if r['prediction'] != 'Error' and r['prediction'] != 'Unknown']
  error_count = len(results) - len(successful_preds)

  logger.info(f"Total claims processed: {len(results)}")
  logger.info(f"Successful predictions: {len(successful_preds)}")
  logger.info(f"Failed predictions: {error_count}")

  if successful_preds:
    pred_counts = {}
    for result in successful_preds:
      pred = result['prediction']
      pred_counts[pred] = pred_counts.get(pred, 0) + 1

    logger.info(f"Prediction distribution: {pred_counts}")

    # Evidence usage analysis
    evidence_counts = []
    for result in successful_preds:
        if 'evidence_used' in result: #
            evidence_counts.append(len(result['evidence_used']))

    if evidence_counts:
        avg_evidence = sum(evidence_counts) / len(evidence_counts)
        logger.info(f"Average evidence pieces per claim: {avg_evidence:.2f}")
        logger.info(f"Evidence range: {min(evidence_counts)} - {max(evidence_counts)} pieces")

    # Tool usage statistics
    tool_info = fact_checker._get_available_tools_info()
    logger.info(f"Tools used: {tool_info}")

    # Basic accuracy calculation
    correct_predictions = sum(1 for r in successful_preds
                          if str(r['prediction']).lower() == str(r['ground_truth']).lower())
    accuracy = correct_predictions / len(successful_preds) if successful_preds else 0
    accuracy_info=f"\nBasic accuracy: {accuracy:.3f} ({correct_predictions}/{len(successful_preds)})"
    logger.info(accuracy_info)

    logger.info(f"\nResults saved to: {output_path}")
    logger.info("Next steps:")
    logger.info("1. Compare with baseline: python src/evaluate.py")
    logger.info("2. Analyze evidence quality and agent reasoning patterns")
    logger.info("3. Generate plots and final report")

  return results




In [10]:
!pip install python-dotenv

In [11]:
from dotenv import load_dotenv
env_path = "/content/drive/MyDrive/env.txt"
load_dotenv(dotenv_path=env_path)
news_api_key=os.getenv("NEWS_API_KEY")
results = run_tool_augmented_experiment(
    news_api_key=news_api_key,
    max_samples=300)

print(f"\nTool-augmented experiment completed! Processed {len(results)} claims.")

INFO:__main__:============================================================
INFO:__main__:STARTING TOOL-AUGMENTED FACT-CHECKING EXPERIMENT
INFO:__main__:============================================================
INFO:__main__:Loading LIAR dataset splits...
INFO:__main__:Loaded training data: 6569 samples
INFO:__main__:Loaded test data: 810 samples
INFO:__main__:
Dataset Statistics:
INFO:__main__:Training set label distribution:
INFO:__main__:binary_label
True     4076
False    2493
Name: count, dtype: int64
INFO:__main__:Test set label distribution:
INFO:__main__:binary_label
True     506
False    304
Name: count, dtype: int64
INFO:__main__:
Initializing tool-augmented fact-checker...
INFO:__main__:This will set up ReACT agent with evidence retrieval capabilities
INFO:__main__:Initializing evidence retrieval tools...
INFO:__main__:Wikipedia tool initialized
INFO:__main__:News API tool initialized
INFO:__main__:Initializing ReACT agent with gpt-3.5-turbo...


/content/drive/MyDrive/DS301_Final_Project/src/utils.py:142: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(model_name=llm_model, temperature=0.2) #!


INFO:__main__:ReACT agent initialized successfully
INFO:__main__:
Starting tool-augmented fact-checking process...
INFO:__main__:Processing first 300 claims for testing


Tool-augmented fact-checking:   0%|          | 0/300 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I need to use the evidence_retrieval tool to search for information on Suzanne Bonamici's stance on a plan that would impact choice for Medicare Advantage seniors.
Action: evidence_retrieval
Action Input: Suzanne Bonamici Medicare Advantage plan
No evidence found for this claim.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I need to try a different search query to find information on Suzanne Bonamici's stance on Medicare Advantage.
Action: evidence_retrieval
Action Input: Suzanne Bonamici stance on Medicare AdvantageEvidence found:
1. Wikipedia: Independent Party of Oregon: The Independent Party of Oregon (IPO) is a centrist political party in the U.S. state of Oregon with more than 140,000 registrants since its inception in January 2007. The IPO is Oregon's third-largest political party and the first political 

...



> Entering new AgentExecutor chain...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I should use the evidence_retrieval tool to search for information on whether Donald Trump suggested negotiating down the national debt.
Action: evidence_retrieval
Action Input: Donald Trump negotiate national debtEvidence found:
1. Wikipedia: Trump Always Chickens Out: Trump Always Chickens Out (TACO), also known as the TACO Trade, is an acronym that gained prominence in May 2025 after many threats and reversals during the trade war Donald Trump initiated with his administration's "Liberation Day" tariffs.
The acronym is used to describe Trump's tendency to make tariff threats, only to later delay them as a way to increase time for negotiations and for markets t...
2. Wikipedia: Business career of Donald Trump: Before running for president of the United States in 2016, Donald Trump pursued a career as a businessman, with a focus on renovating skyscrapers, 

Tool-augmented fact-checking: 100%|█████████▉| 299/300 [1:03:16<00:12, 12.17s/it]



> Entering new AgentExecutor chain...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I should use the evidence_retrieval tool to search for information about the claim regarding media coverage of Donald Trump and another individual.
Action: evidence_retrieval
Action Input: "Donald Trump media coverage"Evidence found:
1. Wikipedia: Public image of Donald Trump: Donald Trump, having previously served as the 45th president and currently as the 47th, has elicited highly polarized public perceptions about his performance as a head of state and largely controversial opinions about his temperament and personal conduct while in office.
Before entering politics, he was a businessman and television personality famous for his image as a real estate tycoon. Viewed ...
2. Wikipedia: Donald Trump's conflict with the media: Donald Trump frequently attacks the American media as having a left-wing bias and being "corrupt", and also uses a variety of insults

Tool-augmented fact-checking: 100%|██████████| 300/300 [1:03:32<00:00, 12.71s/it]

INFO:__main__:Completed tool-augmented fact-checking. Processed 300 claims
INFO:__main__:Saving results to /content/drive/MyDrive/DS301_Final_Project/results/tool_augmented_preds.jsonl
INFO:__main__:======================================================================
INFO:__main__:TOOL-AUGMENTED EXPERIMENT SUMMARY
INFO:__main__:======================================================================
INFO:__main__:Total claims processed: 300
INFO:__main__:Successful predictions: 275
INFO:__main__:Failed predictions: 25
INFO:__main__:Prediction distribution: {'False': 250, 'True': 25}
INFO:__main__:Average evidence pieces per claim: 1.71
INFO:__main__:Evidence range: 0 - 5 pieces
INFO:__main__:Tools used: {'wikipedia': True, 'news_api': False}
INFO:__main__:
Basic accuracy: 0.433 (119/275)
INFO:__main__:
Results saved to: /content/drive/MyDrive/DS301_Final_Project/results/tool_augmented_preds.jsonl
INFO:__main__:Next steps:
INFO:__main__:1. Compare with baseline: python src/evaluate.py
I